In [234]:
library(GEOquery)
library(limma)

In [235]:
gse <- getGEO("GSE19615",GSEMatrix=TRUE)
gse <- gse[[1]]

Found 1 file(s)
GSE19615_series_matrix.txt.gz
Using locally cached version: C:\Users\BIOINF~1\AppData\Local\Temp\RtmpozYrDM/GSE19615_series_matrix.txt.gz
Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)
See spec(...) for full column specifications.
Using locally cached version of GPL570 found here:
C:\Users\BIOINF~1\AppData\Local\Temp\RtmpozYrDM/GPL570.soft 


In [236]:
#exp data
eset <- exprs(gse)
print(eset[1:5,1:5])

          GSM491175 GSM491176 GSM491177 GSM491178 GSM491179
1007_s_at  1211.042   601.809  1373.834   665.438  1316.341
1053_at     219.152   145.972   324.582   119.593   140.175
117_at       92.796   117.997   115.853   148.011   104.725
121_at      228.269   196.550   241.369   265.884   292.212
1255_g_at    13.009    11.365    30.750    33.266    30.740


In [237]:
#probe id to gene symbol
fset <- fData(gse)
colnames(fset) #"Symbol" or "Gene Symbol" using

[1] "ID"                               "GB_ACC"                          
 [3] "SPOT_ID"                          "Species Scientific Name"         
 [5] "Annotation Date"                  "Sequence Type"                   
 [7] "Sequence Source"                  "Target Description"              
 [9] "Representative Public ID"         "Gene Title"                      
[11] "Gene Symbol"                      "ENTREZ_GENE_ID"                  
[13] "RefSeq Transcript ID"             "Gene Ontology Biological Process"
[15] "Gene Ontology Cellular Component" "Gene Ontology Molecular Function"

In [238]:
symbols <- fset[,"Gene Symbol"]
rownames(eset) <- symbols

In [239]:
pset <- phenoData(gse)
colnames(pset)

[1] "title"                                    
 [2] "geo_accession"                            
 [3] "status"                                   
 [4] "submission_date"                          
 [5] "last_update_date"                         
 [6] "type"                                     
 [7] "channel_count"                            
 [8] "source_name_ch1"                          
 [9] "organism_ch1"                             
[10] "characteristics_ch1"                      
[11] "characteristics_ch1.1"                    
[12] "characteristics_ch1.2"                    
[13] "characteristics_ch1.3"                    
[14] "characteristics_ch1.4"                    
[15] "characteristics_ch1.5"                    
[16] "characteristics_ch1.6"                    
[17] "characteristics_ch1.7"                    
[18] "characteristics_ch1.8"                    
[19] "characteristics_ch1.9"                    
[20] "characteristics_ch1.10"                   
[21] "characteristics_ch1.11"                   
[22] "characteristics_ch1.12"                   
[23] "characteristics_ch1.13"                   
[24] "characteristics_ch1.14"                   
[25] "characteristics_ch1.15"                   
[26] "characteristics_ch1.16"                   
[27] "molecule_ch1"                             
[28] "extract_protocol_ch1"                     
[29] "label_ch1"                                
[30] "label_protocol_ch1"                       
[31] "taxid_ch1"                                
[32] "hyb_protocol"                             
[33] "scan_protocol"                            
[34] "description"                              
[35] "data_processing"                          
[36] "platform_id"                              
[37] "contact_name"                             
[38] "contact_institute"                        
[39] "contact_address"                          
[40] "contact_city"                             
[41] "contact_state"                            
[42] "contact_zip/postal_code"                  
[43] "contact_country"                          
[44] "supplementary_file"                       
[45] "data_row_count"                           
[46] "relation"                                 
[47] "adjuvant chemotherapy:ch1"                
[48] "age:ch1"                                  
[49] "chemo class:ch1"                          
[50] "distant recur (yn):ch1"                   
[51] "distant recurrence free survival (mo):ch1"
[52] "er:ch1"                                   
[53] "grade (modified, bloom, richardson):ch1"  
[54] "her.2:ch1"                                
[55] "histology type:ch1"                       
[56] "hormonal rx:ch1"                          
[57] "lymph nodes:ch1"                          
[58] "patientid:ch1"                            
[59] "pr:ch1"                                   
[60] "time of followup (mo):ch1"                
[61] "tissue:ch1"                               
[62] "tumor recurrence (36mo):ch1"              
[63] "tumor size (cm):ch1"

In [240]:
pset3 = pset[pset$'er:ch1' != 'neg' & pset$'hormonal rx:ch1' != 'unknown' & pset$'hormonal rx:ch1' != 'none']
#pset3 = pset[pset$'hormonal rx:ch1' == 'unknown' | pset$'hormonal rx:ch1' == 'none']

eset2 = t(eset)
eset <- eset2[pset$'er:ch1' != 'neg' & pset$'hormonal rx:ch1' != 'unknown' & pset$'hormonal rx:ch1' != 'none',]
#eset <- eset2[pset$'hormonal rx:ch1' == 'unknown' | pset$'hormonal rx:ch1' == 'none',]
eset = t(eset)

In [241]:
dim(eset)

[1] 54675    64

In [242]:
dim(pset3)

sampleNames sampleColumns 
           64            63

In [243]:
grp <- pset3$'tumor recurrence (36mo):ch1'
design <- model.matrix(~0 + grp)
colnames(design)

[1] "grpno"  "grpyes"

In [244]:
colnames(design) <- c("ctrl","recurrence")

In [245]:
#fit
fit <- lmFit(eset,design)
cont <- makeContrasts(recurrence-ctrl,levels=design)
fit.cont <- contrasts.fit(fit,cont)
fit.cont <- eBayes(fit.cont)
res <- topTable(fit.cont,number=Inf)
head(res)

#data output
write.csv(res, file="GSE19615_DEG2.csv")

,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B
24585,,48.78327,19.94194,6.192210,4.994147e-08,0.00273055,-1.523653
14318,PCSK2,131.32228,13.36286,5.670702,3.825518e-07,0.01045801,-1.883121
23323,TUBB2B,315.59378,55.34308,5.502871,7.293901e-07,0.01329313,-2.000623
26850,CPS1,25.84903,22.93328,5.356618,1.273966e-06,0.01741352,-2.103545
26350,MUC7,29.00547,20.69037,4.848334,8.503071e-06,0.07831479,-2.463396
50435,SAE1,21.46207,18.91956,4.845425,8.594216e-06,0.07831479,-2.465458
